In [ ]:
"""
This script is used to calculate the slippage of a client's daily stock exchange.
Though I belive my script is not directly used in the product of our company,
some part of it may contain non-disclosable information of AQUMON, such as the netdisk's folder names.
Therefore, I replaced some non-disclosable information by "#", thank you for inderstanding!
"""
# THE FIRST PART: importing packages and open the documents
import sys
import os
import pandas as pd
import csv
# The following three are package made by AQUMON's quant team
# Fortunately they are not used in the major part of the code
from order_router import OrderRouter
import qtoolkit 
import py_oms_client

path = "/Users/zimu/Documents/AQUMON_intern/account_data_forzimu/"
order_router = OrderRouter(account_dir=os.path.join(path, 'account/'), env_dir=os.path.join(path, 'env'))

# Setting up the display here is to make it easier to see all the data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

# Opening the NAS netdisk's files
file_of_open = open("/Volumes/nfs_share/aqm_collect/###/###/adjust_open.csv"
                    , "r")
file_of_close = open("/Volumes/nfs_share/aqm_collect/###/###/adjust_close.csv"
                     , "r")
file_of_fac = open("/Volumes/nfs_share/aqm_collect/###/###/adjust_factor.csv"
                   , "r")

In [ ]:
# THE SECOND PART: Extract all information from the webcam file. The files need to be opened before this part
# This part should be divided into three functions
# that return three dictionaries containing information about each of the three files.
# Instead of using pandas, I used the csv package, 
# which converts each line of the file into a dict and runs faster than pandas.

# This one is used to extract information about the open price
def extract_open_info(file_of_open):
    # Get all dates here. 
    # You can't use datetime because holidays are not included in the transaction dates. 
    # Globalize this list for later.
    global all_dates
    date_open_info = pd.read_csv(file_of_open, index_col=0)
    all_dates = list(date_open_info.index)
    return date_open_info

In [ ]:
# This one is similar to the function above
# This one is used to extract information about the close price
def extract_close_info(file_of_close):
    date_close_info = pd.read_csv(file_of_close, index_col=0)
    return date_close_info

In [ ]:
# This one is also similar to the function above
# This one is used to extract the information about adjusting factors.
def extract_fac_info(file_of_fac):
    date_fac_info = pd.read_csv(file_of_fac, index_col=0)
    return date_fac_info

In [ ]:
# THE THRID PART: Using pandas to synthesize information
def synthesize_df(date_open_info, date_close_info, date_fac_info, account_name, today_date):
    # Here we are obtaining the date of the previous day
    yesterday_date = all_dates[all_dates.index(today_date) - 1] 

    # Open the following three things separately, order_router needs to be imported in the first section
    today_trans_df = order_router.fetchTransactionHistory(account_name, today_date)
    today_hold_df = order_router.fetchHoldingHistory(account_name, today_date)
    yesterday_hold_df = order_router.fetchHoldingHistory(account_name, yesterday_date)

    trade_time = int(str(list(today_trans_df["filledTime"])[0])[11:13])
    # Here use concat and merge to create synthesized DataFrame and add price, holding, factor information.
    # But it's not enough, the information about the transaction, 
    # such as priceFilled, will involve multiple transactions with the same ID, 
    # resulting in multiple identical indexes, which can't be used with merge, 
    # and will have to be counted individually below.
    synth_df = pd.concat([yesterday_hold_df.to_frame("yesterday_holdings"),
                          today_hold_df.to_frame("today_holdings")], axis=1)
    synth_df = pd.merge(left=synth_df, right=date_open_info.loc[today_date].to_frame("open_price"),
                        left_index=True, right_index=True, how="left")
    synth_df = pd.merge(left=synth_df, right=date_close_info.loc[yesterday_date].to_frame("yesterday_close_price"),
                        left_index=True, right_index=True, how="left")
    synth_df = pd.merge(left=synth_df, right=date_close_info.loc[today_date].to_frame("today_close_price"),
                        left_index=True, right_index=True, how="left")
    synth_df = pd.merge(left=synth_df, right=date_fac_info.loc[yesterday_date].to_frame("yesterday_fac"),
                        left_index=True, right_index=True, how="left")
    synth_df = pd.merge(left=synth_df, right=date_fac_info.loc[today_date].to_frame("today_fac"),
                        left_index=True, right_index=True, how="left")
    synth_df = synth_df.fillna(0)

    # Here we are summing up the fees and volumes of multiple trades with the same ID, 
    # weighting the trades by price, and replacing the trade direction with -1 or 1.
    # Create the following list ready to put data:
    # column of direction, column of commission, column of priceFilled, column of holdingsFilled(or quantityFilled)
    col_of_dire_bi, col_of_commission, col_of_pF, col_of_hF = [], [], [], []
    # Create dicts with IDs corresponding to individual data
    ID_to_wei, ID_to_yes_hol, ID_to_commission, ID_to_direction = {}, {}, {}, {}
    # IDs that made trades today are derived below without repetition
    today_ID = list(set(today_trans_df.index))
    today_ID.sort()
    for ID in today_ID:
        # these three variables are going to increase by each iteration
        weighted = 0
        change = 0
        commission = 0
        try:
            # If multiple transactions are involved, 
            # convert the corresponding IDs and their transaction prices, volumes, and fees into lists.
            every_Pf = list(today_trans_df["priceFilled"].loc[ID])
            every_Hf = list(today_trans_df["quantityFilled"].loc[ID])
            every_commission = list(today_trans_df["commission"].loc[ID])
            # and convert the diraction of teansaction to 1 or -1.
            every_dire = -1 if list(today_trans_df["direction"].loc[ID])[0] == "SELL" else 1
        except:
            # If there is only a single transaction, 
            # it can't be directly converted to a list (because it's not iterable), 
            # but since we're going to use sum(list) below, 
            # we're going to use parentheses on both sides to convert to a list.
            every_Pf = [today_trans_df["priceFilled"].loc[ID]]
            every_Hf = [today_trans_df["quantityFilled"].loc[ID]]
            every_commission = [today_trans_df["commission"].loc[ID]]
            # and convert the diraction of teansaction to 1 or -1.
            every_dire = -1 if [today_trans_df["direction"].loc[ID]][0] == "SELL" else 1
        for i in range(len(every_Hf)):
            weighted += (every_Pf[i] * every_Hf[i]) / sum(every_Hf)
            change += every_Hf[i]
            commission += every_commission[i]
        # Now the weighted price, the volume, the commission, 
        # and the direction of the trade are all out and in the dict
        ID_to_wei[ID] = weighted
        ID_to_yes_hol[ID] = change
        ID_to_commission[ID] = commission
        ID_to_direction[ID] = every_dire
    # Loop over all the IDs in my synthesized DataFrame.
    # Since there may be no transactions that day, 
    # the looped IDs may not be in the transaction history, 
    # in which case the stats will be zero.
    for ID in synth_df.index:
        col_of_pF.append(ID_to_wei[ID] if ID in today_trans_df.index else 0.0)
        col_of_hF.append(ID_to_yes_hol[ID] if ID in today_trans_df.index else 0.0)
        col_of_commission.append(ID_to_commission[ID] if ID in today_trans_df.index else 0.0)
        col_of_dire_bi.append(ID_to_direction[ID] if ID in today_trans_df.index else 0.0)

    # Finally put all four into my synthesized DataFrame,
    # and note that the transaction price is multiplied by the factor.
    # where factor is used to deal with share split that changes the price of stocks.
    synth_df["commission"] = col_of_commission
    synth_df["weighted_priceFilled"] = col_of_pF
    synth_df["quantityFilled"] = col_of_hF
    synth_df["direction_binary"] = col_of_dire_bi
    synth_df["weighted_priceFilled"] = synth_df["weighted_priceFilled"] * synth_df["today_fac"]
    # END OF THE THIRD PART
    return synth_df, trade_time

In [ ]:
# THE FOURTH PART: This part is mostly implementing formula
def calculate_slip_and_pnl(synth_df, trade_time, account_name, today_date):
    # Begin to implement formula
    synth_df["delta_1_slippage"] = (synth_df["direction_binary"]) * synth_df["quantityFilled"] * \
                                   (synth_df["yesterday_close_price"] - synth_df["open_price"]) / synth_df["today_fac"]
    synth_df["delta_1_slippage(%)"] = synth_df["delta_1_slippage"] / (synth_df["yesterday_close_price"]
                                                                      * synth_df["today_holdings"])
    synth_df["delta_2_slippage"] = (synth_df["direction_binary"]) * synth_df["quantityFilled"] * \
                                   (synth_df["open_price"] - synth_df["weighted_priceFilled"]) / synth_df["today_fac"]
    synth_df["delta_2_slippage(%)"] = synth_df["delta_2_slippage"] / (synth_df["yesterday_close_price"]
                                                                      * synth_df["today_holdings"])
    # The first formula is the formula for transaction slippage
    if trade_time < 14:
        synth_df["trans_slippage"] = (synth_df["direction_binary"]) * synth_df["quantityFilled"] / \
                                     synth_df["today_fac"] * \
                                     (synth_df["yesterday_close_price"] - synth_df["weighted_priceFilled"])
        synth_df["trans_slippage(%)"] = synth_df["trans_slippage"] / (synth_df["yesterday_close_price"]
                                                                      * synth_df["today_holdings"])
    else:
        # If you got it after 14:00 pm, then replace yeaterday_close with today_close
        synth_df["trans_slippage"] = (synth_df["direction_binary"]) * synth_df["quantityFilled"] / \
                                     synth_df["today_fac"] * \
                                     (synth_df["today_close_price"] - synth_df["weighted_priceFilled"])
        synth_df["trans_slippage(%)"] = synth_df["trans_slippage"] / (synth_df["yesterday_close_price"]
                                                                      * synth_df["today_holdings"])
    # The second formula is the formula for pnl_real (real PnL)
    # pnl refers to "Profit and Loss"
    synth_df["pnl_real"] = synth_df["yesterday_holdings"] * \
                           (synth_df["weighted_priceFilled"] - synth_df["yesterday_close_price"]) / \
                           synth_df["yesterday_fac"]+\
                           synth_df["today_holdings"] * \
                           (synth_df["today_close_price"] - synth_df["weighted_priceFilled"]) / synth_df["today_fac"]
    synth_df["pnl_real(%)"] = synth_df["trans_slippage"] / (synth_df["yesterday_close_price"]
                                                            * synth_df["today_holdings"])

    # The third formula is the formula for pnl_naive (naive PnL)
    # pnl refers to "Profit and Loss"
    synth_df["pnl_naive"] = synth_df["today_holdings"] * \
                            (synth_df["today_close_price"] - synth_df["yesterday_close_price"]) / synth_df["today_fac"]
    synth_df["pnl_naive(%)"] = synth_df["trans_slippage"] / (synth_df["yesterday_close_price"]
                                                             * synth_df["today_holdings"])
    # Lastly, calculate the turnover
    synth_df["trade_volume"] = synth_df["quantityFilled"] * synth_df["weighted_priceFilled"] / synth_df["today_fac"]
    synth_df["turnover"] = synth_df["trade_volume"] / (synth_df["yesterday_close_price"] * synth_df["today_holdings"])
    # get the last 12 columns, which I want to use.
    # The name for those columns will be defined later
    calculated_results_df = synth_df.iloc[:, -12:]
    yesterday_date = all_dates[all_dates.index(today_date) - 1]
    overall_cur_pv = order_router.fetchPVHistory(account_name, yesterday_date)
    overall_delta_1 = sum(list(synth_df["delta_1_slippage"]))
    overall_delta_2 = sum(list(synth_df["delta_2_slippage"]))
    overall_trans_slip = sum(list(synth_df["trans_slippage"]))
    overall_pnl_real = sum(list(synth_df["pnl_real"]))
    overall_pnl_naive = sum(list(synth_df["pnl_naive"]))
    overall_trade_volume = sum(list(synth_df["trade_volume"]))
    calculated_results_df.loc["overall"] = [f"{overall_delta_1:.4f}",
                                            f"{overall_delta_1 / overall_trade_volume:.4%}",
                                            f"{overall_delta_2:.4f}",
                                            f"{overall_delta_2 / overall_trade_volume:.4%}",
                                            f"{overall_trans_slip:.4f}",
                                            f"{overall_trans_slip / overall_trade_volume:.4%}",
                                            f"{overall_pnl_real:.4f}",
                                            f"{overall_pnl_real / overall_cur_pv:.4%}",
                                            f"{overall_pnl_naive:.4f}",
                                            f"{overall_pnl_naive / overall_cur_pv:.4%}",
                                            f"{overall_trade_volume:.4f}",
                                            f"{overall_trade_volume / overall_cur_pv:.4%}"]
    return calculated_results_df

In [ ]:
# Finally, write a csv file that covers all the data you want to count for the selected date range
def produce_result_csv(account_name, start_date, end_date, open_info, close_info, fac_info):
    produced_file = open(f"slippage_and_pnl_of_{account_name}.csv", "wt")
    # Write out the first line of the header
    produced_file.write("Dates,"+",".join(["delta_1($)", "delta_1(%)", "delta_2($)", "delta_2(%)",
                                           "trans_slippage($)", "trans_slippage(%)", "pnl_real($)", "pnl_real(%)",
                                           "pnl_naive($)", "pnl_naive(%)", "trade_volume", "turnover"])+"\n")
    # Write up the data for each date
    for dates in all_dates[all_dates.index(start_date): all_dates.index(end_date)+1]:
        df, tradetime = synthesize_df(open_info, close_info, fac_info, account_name, dates)
        final_df = calculate_slip_and_pnl(df, tradetime, account_name, dates)
        produced_file.write(dates+", "+", ".join(list(final_df.loc["overall"]))+"\n")

In [ ]:
open_info = extract_open_info(file_of_open)
close_info = extract_close_info(file_of_close)
fac_info = extract_fac_info(file_of_fac)
produce_result_csv("######_momt", "2022-##-##", "2022-##-##", open_info, close_info, fac_info)

file_of_fac.close()
file_of_close.close()
file_of_fac.close()